# Ajuste de hiperparámetros con un trabajo de barrido

Hay muchos algoritmos de aprendizaje automático que necesitan hiperparámetros (valores de parámetro que influyen en el entrenamiento, pero que no se pueden determinar a partir de los propios datos de entrenamiento). Por ejemplo, al entrenar un modelo de regresión logística, puede usar un hiperparámetro de tasa de regularización para contrarrestar el sesgo en el modelo; o bien al entrenar una red neuronal convolucional, puede usar hiperparámetros como velocidad de aprendizaje y tamaño de lote para controlar cómo se ajustan los pesos y cuántos elementos de datos se procesan en un mini lote, respectivamente. La elección de valores de hiperparámetro puede afectar significativamente al rendimiento de un modelo entrenado o al tiempo que se necesita para entrenarlo, y a menudo es necesario probar varias combinaciones para encontrar la solución óptima. 

## Antes de comenzar

Necesitará la versión más reciente del paquete **azureml-ai-ml** para ejecutar el código en este cuaderno. Ejecute la celda siguiente para comprobar que está instalada.

> **Nota**:
> Si el paquete **azure-ai-ml** no está instalado, ejecute `pip install azure-ai-ml` para instalarlo.

In [ ]:
## Conexión con su área de trabajo

Con los paquetes de SDK necesarios instalados, ya está listo para conectarse al área de trabajo.

Para conectarse a un espacio de trabajo, se necesitan parámetros de identificación: un id. de suscripción, un nombre de grupo de recursos y un nombre de espacio de trabajo. El nombre del grupo de recursos y el nombre del área de trabajo ya se rellenan automáticamente. Solo necesita el identificador de suscripción para completar el comando.

Para buscar los parámetros necesarios, haga clic en la suscripción y el nombre del área de trabajo en la parte superior derecha de Studio. Se abrirá un panel a la derecha.

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> Copie el identificador de suscripción y reemplace **YOUR-SUBSCRIPTION-ID** por el valor que copió. </p>

## Creación del script de entrenamiento
El ajuste de hiperparámetros es ideal cuando desea entrenar modelos de aprendizaje automático, pero variar los parámetros de entrada. Deberá crear un script de entrenamiento que espere un parámetro de entrada que represente uno de los hiperparámetros del algoritmo.

Ejecute las siguientes celdas para crear la carpeta **src** y el script de entrenamiento.

Tenga en cuenta que el script de entrenamiento espera dos parámetros de entrada:

- `--training_data` que espera una cadena. Especificará la ruta de acceso a un recurso de datos registrado como datos de entrenamiento de entrada.
- `--reg_rate` que espera un número, pero tiene un valor predeterminado de `0.01`. Usará este parámetro de entrada para el ajuste de hiperparámetros.

In [ ]:
## Configuración y ejecución de un trabajo de comando

Ejecute la celda siguiente para entrenar un modelo de clasificación para predecir la diabetes. El modelo se entrena ejecutando el script **train\.py** que se encuentra en la carpeta **src**. Utiliza el activo de datos registrado `diabetes-data` como datos de entrenamiento. 

- `code`: especifica la carpeta que incluye el script a ejecutar.
- `command`: especifica qué ejecutar exactamente.
- `environment`: especifica los paquetes necesarios que se van a instalar en el proceso antes de ejecutar el comando.
- `compute`: especifica el proceso que se va a usar para ejecutar el comando.
- `display_name`: nombre del trabajo individual.
- `experiment_name`: nombre del experimento al que pertenece el trabajo.

Tenga en cuenta que el trabajo de comando solo ejecuta el script de entrenamiento una vez, con una tasa de regularización de `0.1`. Antes de ejecutar un trabajo de barrido para ajustar hiperparámetros, se recomienda probar si el script funciona según lo previsto con un trabajo de comando.

In [ ]:
## Definición del espacio de búsqueda

Cuando el trabajo de comando se haya completado correctamente, puede configurar y ejecutar un trabajo de barrido. 

En primer lugar, deberá especificar el espacio de búsqueda para el hiperparámetro. Para entrenar tres modelos, cada uno con una tasa de regularización diferente (`0.01`, `0.1` o `1`), puede definir el espacio de búsqueda con un hiperparámetro `Choice`. 

## Configuración y envío del trabajo de barrido

Usará la función de barrido para realizar el ajuste de hiperparámetros en el script de entrenamiento. Para configurar un trabajo de barrido, deberá configurar lo siguiente:

- `compute`: nombre del destino de proceso en el que se ejecutará el trabajo.
- `sampling_algorithm`: algoritmo de muestreo de hiperparámetros que se va a usar en el espacio de búsqueda. Los valores permitidos son `random`, `grid` y `bayesian`.
- `primary_metric`: el nombre de la métrica principal notificada por cada trabajo de prueba. La métrica debe registrarse en el script de entrenamiento del usuario mediante `mlflow.log_metric()` con el mismo nombre de métrica correspondiente.
- `goal`: El objetivo de optimización del `primary_metric`. Los valores permitidos son `maximize` y `minimize`.
- `limits`: límites del trabajo de barrido. Por ejemplo, la cantidad máxima de pruebas o modelos que desea entrenar.

Tenga en cuenta que el trabajo de comando se usa como base para el trabajo de barrido. El trabajo de barrido reutilizará la configuración del trabajo de comando.

In [ ]:
Ejecute la celda siguiente para enviar el trabajo de barrido.

In [ ]:
Una vez completado el trabajo, vaya a la información general del trabajo. En la pestaña **Pruebas** se mostrarán todos los modelos entrenados y cómo difiere la puntuación `Accuracy` para cada valor de tasa de regularización que haya probado.

## Configure and run a command job

Run the cell below to train a classification model to predict diabetes. The model is trained by running the **train\.py** script that can be found in the **src** folder. It uses the registered `diabetes-data` data asset as the training data. 

- `code`: specifies the folder that includes the script to run.
- `command`: specifies what to run exactly.
- `environment`: specifies the necessary packages to be installed on the compute before running the command.
- `compute`: specifies the compute to use to run the command.
- `display_name`: the name of the individual job.
- `experiment_name`: the name of the experiment the job belongs to.

Note that the command job only runs the training script once, with a regularization rate of `0.1`. Before you run a sweep job to tune hyperparameters, it's a best practice to test whether your script works as expected with a command job.

In [ ]:
from azure.ai.ml import command, Input
from azure.ai.ml.constants import AssetTypes

# configure job

job = command(
    code="./src",
    command="python train.py --training_data ${{inputs.diabetes_data}} --reg_rate ${{inputs.reg_rate}}",
    inputs={
        "diabetes_data": Input(
            type=AssetTypes.URI_FILE, 
            path="azureml:diabetes-data:1"
            ),
        "reg_rate": 0.01,
    },
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest",
    compute="aml-cluster",
    display_name="diabetes-train-mlflow",
    experiment_name="diabetes-training", 
    tags={"model_type": "LogisticRegression"}
    )

# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

## Define the search space

When your command job has completed successfully, you can configure and run a sweep job. 

First, you'll need to specify the search space for your hyperparameter. To train three models, each with a different regularization rate (`0.01`, `0.1`, or `1`), you can define the search space with a `Choice` hyperparameter. 

In [ ]:
from azure.ai.ml.sweep import Choice

command_job_for_sweep = job(
    reg_rate=Choice(values=[0.01, 0.1, 1]),
)

## Configure and submit the sweep job

You'll use the sweep function to do hyperparameter tuning on your training script. To configure a sweep job, you'll need to configure the following:

- `compute`: Name of the compute target to execute the job on.
- `sampling_algorithm`: The hyperparameter sampling algorithm to use over the search space. Allowed values are `random`, `grid` and `bayesian`.
- `primary_metric`: The name of the primary metric reported by each trial job. The metric must be logged in the user's training script using `mlflow.log_metric()` with the same corresponding metric name.
- `goal`: The optimization goal of the `primary_metric`. The allowed values are `maximize` and `minimize`.
- `limits`: Limits for the sweep job. For example, the maximum amount of trials or models you want to train.

Note that the command job is used as the base for the sweep job. The configuration for the command job will be reused by the sweep job.

In [ ]:
# apply the sweep parameter to obtain the sweep_job
sweep_job = command_job_for_sweep.sweep(
    compute="aml-cluster",
    sampling_algorithm="grid",
    primary_metric="training_accuracy_score",
    goal="Maximize",
)

# set the name of the sweep job experiment
sweep_job.experiment_name="sweep-diabetes"

# define the limits for this sweep
sweep_job.set_limits(max_total_trials=4, max_concurrent_trials=2, timeout=7200)

Run the following cell to submit the sweep job.

In [ ]:
returned_sweep_job = ml_client.create_or_update(sweep_job)
aml_url = returned_sweep_job.studio_url
print("Monitor your job at", aml_url)

When the job is completed, navigate to the job overview. The **Trials** tab will show all models that have been trained and how the `Accuracy` score differs for each regularization rate value you tried.